## Static Variables

In [6]:
class Parent:
    order = 1
    def __init__(self):
        self.ins_variable1 = 10

class Parent2:
    order = 1
    def __init__(self):
        self.ins_variable1 = 10
        self.order = 20  # shadows static variable if accessed via instance reference
        
p = Parent()
print(p.order, Parent.order)

p2 = Parent2()
print(p2.order, Parent2.order)

1 1
20 1


In [8]:
class Child:
    order = 2
    def __init__(self):
        self.ins_variable1 = 20
        
c = Child()
print(c.order, Child.order, Parent.order)


2 2 1


## `getattr()` method

The getattr() method returns the value of the named attribute of an object. If not found, it returns the default value provided to the function.

In [1]:
class Person:
    age = 23
    name = "Adam"

person = Person()

# when default value is provided
print('The sex is:', getattr(person, 'sex', 'Male'))

# when no default value is provided
print('The sex is:', getattr(person, 'sex'))

The sex is: Male


AttributeError: 'Person' object has no attribute 'sex'

# Meta Classes

https://realpython.com/python-metaclasses/

Remember that, in Python, everything is an object. Classes are objects as well. As a result, a class must have a type. What is the type of a class?

In [1]:
class Foo:
    pass

x = Foo()

print(type(x))
print(type(Foo))

for t in int, float, dict, list, tuple:
    print(type(t))

<class '__main__.Foo'>
<class 'type'>


In the above case:

![](https://files.realpython.com/media/class-chain.5cb031a299fe.png)


* x is an instance of class Foo.
* Foo is an instance of the type metaclass.
* type is also an instance of the type metaclass, so it is an instance of itself.

You can also call type() with three arguments—type(`<name>, <bases>, <dct>`):

* `<name>` specifies the class name. This becomes the `__name__` attribute of the class.
* `<bases>` specifies a tuple of the base classes from which the class inherits. This becomes the `__bases__` attribute of the class.
* `<dct>` specifies a namespace dictionary containing definitions for the class body. This becomes the `__dict__` attribute of the class.

In [3]:
Foo = type('Foo', (), {})
x = Foo()
x

In [5]:
Bar = type('Bar', (Foo,), dict(attr=100, attr_val=lambda self : self.attr))

x = Bar()
print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)
x.attr_val()

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


100

## Custom Meta Class

In [8]:
class Foo:
    def __init__(self):
        print("Init is called")
        
    pass

f = Foo()

Init is called


The expression Foo() creates a new instance of class Foo. When the interpreter encounters Foo(), the following occurs:

The `__call__()` method of Foo’s parent class is called. Since Foo is a standard new-style class, its parent class is the type metaclass, so type’s `__call__()` method is invoked.

That `__call__()` method in turn invokes the following:

* `__new__()`
* `__init__()`

If Foo does not define `__new__()` and `__init__()`, default methods are inherited from Foo’s ancestry. But if Foo does define these methods, they override those from the ancestry, which allows for customized behavior when instantiating Foo.

In the following, a custom method called new() is defined and assigned as the `__new__()` method for Foo:

In [10]:
def new(cls):
    print("new is called")
    x = object.__new__(cls)
    x.attr = 100
    return x

Foo.__new__ = new  # This is Foo class's new method to create objects

f = Foo()
print(f.attr)


g = Foo()
print(g.attr)

new is called
Init is called
100
new is called
Init is called
100


This modifies the instantiation behavior of class Foo: each time an instance of Foo is created, by default it is initialized with an attribute called attr, which has a value of 100. (Code like this would more usually appear in the `__init__()` method and not typically in `__new__()`. This example is contrived for demonstration purposes.)

Now, as has already been reiterated, classes are objects too. Suppose you wanted to similarly customize instantiation behavior when creating a class like Foo. If you were to follow the pattern above, you’d again define a custom method and assign it as the `__new__()` method for the class of which Foo is an instance. Foo is an instance of the type metaclass, so the code looks something like this:

In [11]:
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

Except, as you can see, you can’t reassign the `__new__()` method of the type metaclass. Python doesn’t allow it.

This is probably just as well. type is the metaclass from which all new-style classes are derived. You really shouldn’t be mucking around with it anyway. But then what recourse is there, if you want to customize instantiation of a class?

One possible solution is a custom metaclass. Essentially, instead of mucking around with the type metaclass, you can define your own metaclass, which derives from type, and then you can muck around with that instead.

The first step is to define a metaclass that derives from type, as follows:

In [13]:
class Meta(type):
    def __new__(cls, name, bases, dct):  # This is meta classes new method
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

The definition header class Meta(type): specifies that Meta derives from type. Since type is a metaclass, that makes Meta a metaclass as well.

Note that a custom `__new__()` method has been defined for Meta. It wasn’t possible to do that to the type metaclass directly. The `__new__()` method does the following:

* Delegates via super() to the `__new__()` method of the parent metaclass (type) to actually create a new class
* Assigns the custom attribute attr to the class, with a value of 100
* Returns the newly created class

Now the other half of the voodoo: Define a new class Foo and specify that its metaclass is the custom metaclass Meta, rather than the standard metaclass type. This is done using the metaclass keyword in the class definition as follows:

In [15]:
class Foo(metaclass=Meta):
    pass

class Bar(metaclass=Meta):
    pass

class Qux(metaclass=Meta):
    pass

print(Foo.attr, Bar.attr, Qux.attr)

100 100 100


In the same way that a class functions as a template for the creation of objects, a metaclass functions as a template for the creation of classes. Metaclasses are sometimes referred to as class factories.

In [16]:
class Foo:
    def __init__(self):
        self.attr = 100

x = Foo()
y = Foo()
z = Foo()

print(x.attr, y.attr, z.attr)

100 100 100


In [17]:
class Meta(type):
    def __init__(  # Notice that this is __init__, unlike __new__
        cls, name, bases, dct
    ):
        cls.attr = 100

class X(metaclass=Meta):
    pass

class Y(metaclass=Meta):
    pass

class Z(metaclass=Meta):
    pass

print(X.attr, Y.attr, Z.attr)

100 100 100


## Is This Really Necessary?
As simple as the above class factory example is, it is the essence of how metaclasses work. They allow customization of class instantiation.

Still, this is a lot of fuss just to bestow the custom attribute attr on each newly created class. Do you really need a metaclass just for that?

In Python, there are at least a couple other ways in which effectively the same thing can be accomplished:

In [20]:
# Using Simple Inheritance

class Base:
    attr = 100


class X(Base):
    pass


class Y(Base):
    pass


class Z(Base):
    pass

print(X.attr, Y.attr, Z.attr)

100 100 100


In [19]:
# Using decorators

def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class X:
    pass

@decorator
class Y:
    pass

@decorator
class Z:
    pass


print(X.attr, Y.attr, Z.attr)

100 100 100
